In [1]:
import pandas as pd

In [2]:

# dataf = pd.read_csv("/mnt/lab_data2/anusri/chrombpnet/src/evaluation/figure_notebooks/figure_6/final_figures_new/smc.caqtls.tsv", sep="\t", header=0)
# dataf
# sum(dataf["IsUsed"])
# dataf.columns

In [3]:
orig = pd.read_csv("/mnt/lab_data2/anusri/variant-scorer/src/output/smc_caqtls/SMC_rasqual_results_FDR_0.05_083121.txt",  sep="\t", header=0)
preds = pd.read_csv("/mnt/lab_data2/anusri/variant-scorer/src/output/smc_caqtls/ATAC_sig_only_new/summary.mean.variant_scores_new.tsv", sep="\t", header=0)
import numpy as np
preds["jsd.sign.mean"] = np.sign(preds["logfc.mean"])* preds["jsd.mean"]
preds = preds[["chr", "pos", "allele1", "allele2", "logfc.mean", "jsd.sign.mean"]]
orig = orig.merge(preds, left_on=['Chromosome', 'SNP_position', 'Ref_allele','Alt_allele'], right_on=["chr", "pos", "allele1", "allele2"]).drop_duplicates()
orig["pos0"] = orig["SNP_position"] - 1
selected_beds = orig[['Chromosome', "pos0", 'SNP_position', 'Ref_allele', 'Alt_allele', 'Log10_BH_Q', 'Effect_size', 'logfc.mean', 'jsd.sign.mean']]


fold_0=pd.read_csv("/oak/stanford/groups/akundaje/anusri/chrombpnet_zenodo_uploads_reformatted/SMC_scATAC_ChromBPNet/data/fold_0/peaks.testset.bed.gz", sep="\t", header=None)
fold_1=pd.read_csv("/oak/stanford/groups/akundaje/anusri/chrombpnet_zenodo_uploads_reformatted/SMC_scATAC_ChromBPNet/data/fold_1/peaks.testset.bed.gz", sep="\t", header=None)
fold_2=pd.read_csv("/oak/stanford/groups/akundaje/anusri/chrombpnet_zenodo_uploads_reformatted/SMC_scATAC_ChromBPNet/data/fold_2/peaks.testset.bed.gz", sep="\t", header=None)
fold_3=pd.read_csv("/oak/stanford/groups/akundaje/anusri/chrombpnet_zenodo_uploads_reformatted/SMC_scATAC_ChromBPNet/data/fold_3/peaks.testset.bed.gz", sep="\t", header=None)
fold_4=pd.read_csv("/oak/stanford/groups/akundaje/anusri/chrombpnet_zenodo_uploads_reformatted/SMC_scATAC_ChromBPNet/data/fold_4/peaks.testset.bed.gz", sep="\t", header=None)
peaks_bed = pd.concat([fold_0, fold_1, fold_2, fold_3, fold_4])
peaks_bed[1] = peaks_bed[1] + peaks_bed[9] - 100
peaks_bed[2] = peaks_bed[1] + 200


import pybedtools
beds_var = pybedtools.BedTool.from_dataframe(selected_beds)
peak_bed =  pybedtools.BedTool.from_dataframe(peaks_bed)
sig_vars_in_bed = beds_var.intersect(peak_bed, c=True)

sig_vars_in_bed = sig_vars_in_bed.to_dataframe(names=['Chromosome', "pos0", 'SNP_position', 'Ref_allele', 'Alt_allele', 'Log10_BH_Q', 'Effect_size', 'logfc.mean', 'jsd.sign.mean', "hits"]).drop_duplicates()


sig_vars_in_bed["IsUsed"] = sig_vars_in_bed["hits"] > 0

In [4]:
sig_vars_in_bed

,Chromosome,pos0,SNP_position,Ref_allele,Alt_allele,Log10_BH_Q,Effect_size,logfc.mean,jsd.sign.mean,hits,IsUsed
0,chr1,1067006,1067007,A,G,-4.920961,0.200715,0.022735,0.008771,0,False
1,chr1,108121218,108121219,C,G,-4.118572,0.228169,0.021728,0.009595,0,False
2,chr1,108721446,108721447,C,T,-9.266900,0.764114,0.289341,0.031146,1,True
3,chr1,108842307,108842308,G,C,-4.721082,0.242912,-0.293381,-0.041953,0,False
4,chr1,109272629,109272630,A,G,-3.006430,0.709092,0.100375,0.016716,0,False
...,...,...,...,...,...,...,...,...,...,...,...
1979,chr9,96159156,96159157,C,T,-3.026381,0.806026,-0.104170,-0.024419,0,False
1980,chr9,96178224,96178225,G,A,-4.596132,0.787369,0.016330,0.007556,0,False
1981,chr9,97227899,97227900,A,T,-2.961709,0.405945,-0.000896,-0.007822,0,False
1982,chr9,98873906,98873907,T,C,-4.182089,0.608833,-0.040235,-0.008717,0,False


In [5]:
sum(sig_vars_in_bed["IsUsed"])

386

In [6]:
sig_vars_in_bed.columns

Index(['Chromosome', 'pos0', 'SNP_position', 'Ref_allele', 'Alt_allele',
       'Log10_BH_Q', 'Effect_size', 'logfc.mean', 'jsd.sign.mean', 'hits',
       'IsUsed'],
      dtype='object')

In [7]:
orig.columns

Index(['Feature_ID', 'rsID', 'Chromosome', 'SNP_position', 'Ref_allele',
       'Alt_allele', 'Freq', 'HWE_Chisquare', 'Imp_quality', 'Log10_BH_Q',
       'Chisquare', 'Effect_size', 'Delta', 'Phi', 'Overdispersion',
       'SNP_id_region', 'Num_feature_SNPs', 'Num_tested_SNPs',
       'Num_iterations_null', 'Num_iterations_alt', 'Random_ties',
       'Log_likelihood_null', 'Convergence_status', 'Sq_corr_fSNPs',
       'Sq_corr_rSNP', 'P', 'q', 'keep', 'PeakID', 'Chr', 'Start', 'End',
       'Strand', 'Peak Score', 'Focus Ratio/Region Size', 'Annotation',
       'Detailed Annotation', 'Distance to TSS', 'Nearest PromoterID',
       'Entrez ID', 'Nearest Unigene', 'Nearest Refseq', 'Nearest Ensembl',
       'Gene Name', 'Gene Alias', 'Gene Description', 'Gene Type', 'chr',
       'pos', 'allele1', 'allele2', 'logfc.mean', 'jsd.sign.mean', 'pos0'],
      dtype='object')

In [8]:
sig_vars_in_bed.shape

(1984, 11)

In [9]:
orig.shape

(1984, 54)

In [10]:
rename_dict = {'Chromosome': "var.chr",
                'SNP_position': "var.pos_hg38",
                'Ref_allele': "var.ref",
                'Alt_allele': "var.alt",
                'Log10_BH_Q': "obs.Log10_BH_Q", 
                'Effect_size': "obs.Effect_size",
                'logfc.mean': "pred.chrombpnet.smc.varscore.logfc",
                'jsd.sign.mean': "pred.chrombpnet.smc.varscore.jsd",
                'IsUsed': "var.isused"}

In [11]:
data = sig_vars_in_bed.rename(columns=rename_dict)

In [12]:
data.columns

Index(['var.chr', 'pos0', 'var.pos_hg38', 'var.ref', 'var.alt',
       'obs.Log10_BH_Q', 'obs.Effect_size',
       'pred.chrombpnet.smc.varscore.logfc',
       'pred.chrombpnet.smc.varscore.jsd', 'hits', 'var.isused'],
      dtype='object')

In [13]:
use_cols=['var.chr', 'pos0', 'var.pos_hg38', 'var.ref', 'var.alt',
       'obs.Log10_BH_Q', 'obs.Effect_size',
       'pred.chrombpnet.smc.varscore.logfc',
       'pred.chrombpnet.smc.varscore.jsd', 'var.isused']

In [14]:
dataf = data[use_cols]


In [15]:
dataf_final = dataf[dataf["var.isused"]]
dataf_final = dataf_final.drop(["var.isused"], axis=1)
dataf.to_csv("full_variants/caqtls.smc.benchmarking.all.tsv", sep='\t', header=True, index=False)
dataf_final.to_csv("filtered_variants/caqtls.smc.benchmarking.chrombpnet.tsv", sep='\t', header=True, index=False)

In [16]:
dataf_final.shape

(386, 9)

In [17]:
dataf.shape

(1984, 10)